# Wprowadzenie

<div class="alert alert-block alert-success">
    <p>Niniejszy program pobiera plik <i>tldr.txt</i> z folderu użytkownika Pobieranie i przekształca go na użyteczną bazę danych. </p>
    <p>Do użytku są dwie funkcjie <b>szukanieKc()</b> oraz <b>karty_od_uzytkownika()</b></p>
    <p>Więcej szczegółów poniżej</p>
</div>

<div class="alert alert-block alert-warning">
    <p>Pamiętaj aby wrzucić tu swój plik <b>tldr.txt</b> inaczej program nie zadziała.</p>
    <p>Aby użyć <b>jakiejkolwiek funkcji</b> najpierw należy uruchomić cały program by przekonwertował dane.</p>
    <p>W przeciwnym wypadku format tekstu nie będzie się zgadzał i żadna z funkcji nie zadziała</p>
</div>

<div class="alert alert-block alert-danger">
    <div  style="text-align:center">
        <h1>!! Uwaga !!</h1>
            <font size=3>Nie zmieniaj parametrów po nazwie użytkownia w poniższej funkji!</font>
        </div>
</div>

<font size=5> S.wl <font color='green'>JarMar</font> nie tak tak tak tak</font>

In [16]:
import os
import re
import pandas as pd

data_list = []

with open('tldr.txt', mode='r', encoding='utf8') as plik:
    current_data = None

    for x in plik:
        if x.startswith('????', 3) or x.startswith('\n'):
            continue
        if x.startswith(':', 3):
            continue
        if x.startswith('**'):
            if current_data:
                data_list.append(current_data)

            name_match = re.search(r'\[([^\]]+)\]', x)
            kc_match = re.search(r'KC:\s*(\d+)', x)
            id_match = re.search(r'character/\s*(\d+)', x)
            name = name_match.group(1) if name_match else 'Unknown'
            kc = kc_match.group(1) if kc_match else 'Unknown'
            index = id_match.group(1) if id_match else 'Unknown'
            current_data = {
                "name": str(name),
                'ID': int(index),
                "kc": int(kc),
                "users": []
            }
        else:
            if x.count(']')>2:
                nick_match = re.search(r'\[(.*?)\]', x)
                if nick_match and current_data:
                    nick = nick_match.group(1)
                    current_data["users"].append(nick)
                else:
                    nick_match = re.search(r'^\d+:\s+([^:\s]+)', x)
                    if nick_match and current_data:
                        nick = nick_match.group(1)
                        current_data["users"].append(nick)
    if current_data:
        data_list.append(current_data)

with open('tldr_altered.txt', mode='w', encoding='utf8') as nowy_plik:
    for i, data in enumerate(data_list, 1):
        line = '_' * 50 + '\n'
        nowy_plik.write(line)
        nowy_plik.write(f"#{i} | {data['name']} | ID: {data['ID']} | kc: {data['kc']} | posiada: {len(data['users'])}\n")
        nowy_plik.write('\n')
        
        for user in data["users"]:
            nowy_plik.write(user + '\n')
print('Plik tekstowy jest już gotowy')

Plik tekstowy jest już gotowy


In [7]:
df = pd.DataFrame.from_dict(data_list)
df['name']=df['name'].astype(str)
df['kc']=df['kc'].convert_dtypes(convert_integer=True)
df['ID']=df['ID'].convert_dtypes(convert_integer=True)

Wyświetlam pierwsze 5 wierszy.

In [8]:
df.head()

,name,ID,kc,users
0,Rias Gremory,23436,56,"[Laudemort, laros3, Kadzee_, DareQPlays]"
1,Ayato Kirishima,68334,12,"[MasterUnit, pekorella, Madcatlover, ReTonZaRR..."
2,Q,50742,1,"[Hurkisq, KrychuXIV, lolcrafter1100, Zielakos,..."
3,Eustass Kid,13620,4,"[paula989, PyraZiemniaczana, Ingsay, dragon8, ..."
4,Musashi,3824,3,"[Sniku, SzymonWidziszowski, GoldfishBrain, Kal..."


Wyświetlam pierwsze 10 pozycji posortowanych malejąco po KC

In [9]:
df.sort_values('kc',ascending=False).head(10).reset_index(drop=True)

,name,ID,kc,users
0,Rias Gremory,23436,56,"[Laudemort, laros3, Kadzee_, DareQPlays]"
1,Satoru Gojou,97696,54,"[Hatchkey, Krystiandfrhgsenpai, Ema23, Hurkisq..."
2,Nezuko Kamado,82447,47,"[Ema23, Teecari, proskondzio, JestJakJest, Gol..."
3,Yue,79932,46,"[XMYS, Otoczaku, Crayonami, Senpai_, Shadow202..."
4,Esdeath,33598,45,"[Kali2111, Krasnest, Loovi, Kryspek, tomasikgu..."
5,Asuna Yuuki,45276,44,"[kacper204100, Krysana, Wjerzba, Idkidei, Talu..."
6,Ken Kaneki,58123,41,"[DarGorN, ringoameyuri, Kaczorpl3, Hurkisq]"
7,Yuuji Itadori,97667,41,"[Misterx, iza_kicia, Ingsay, LargeMonster, Hat..."
8,Sukuna Ryoumen,102976,40,"[Ema23, Partyganiusz, Oliwianin, Okigai]"
9,Eris Boreas Greyrat,69544,40,"[Sebal103, Losingxcontrol, korendan, Rtopiroks..."


# Pierwsza Funkcja
<div class="alert alert-block alert-info">
    <p>Pierwsza funckcja <b>szukanieKc()</b> wymaga dwóch parametrów by zadziałać. Funkcja zwraca listę kart z wybranego przedziału KC który nas interesuje.</p>
    <p>Jeżeli nie chcemy bądź nie wiemy jak wiele jakaś karta ma KC możemy wpisać <b>gora<font color='purple'>=</font><font color='green'>None</font></b>. W ten sposób funkcja bedzie ograniczona tylko od dołu</p>
</div>

In [10]:
def szukanieKc(dol=None, gora=None):
    '''
    
    dol - dolny przedział w jakim funkcja ma szukać kart
    gora - górny przedział do jakiego funkcja zwróci karty
    
    ---------
    przykład:
    
    szukanieKc(10, 15)
    przedział [10, 15] domknięty obóstronnie.
    Funkcja zwróci listę wszystkich graczy którzy posiadają daną kartę o pożądanym KC
    
    Możliwe jest niepodanie górnego bądź dolnego przedziału, 
    wówczas funkcja wybierze maksymalne bądź minimalne wartosci.
    
    '''
    if dol == None:
        dol=df['kc'].min()
    if gora == None:
        gora = df['kc'].max()
        
    question = df[(df['kc']<=gora)&(df['kc']>=dol)].sort_values("kc",ascending=False)

    print('Znaleziono ', len(question), f' kart z przedziału [{dol}, {gora}]')
    for i in range(len(question)):
        print('_'*30,)
        print('\n>',f'{i+1}.', question.iloc[i]['name'], question.iloc[i]['kc'], 'kc <\n')
        for j in range(len(question.iloc[i]['users'])):
            print('     '+question.iloc[i]['users'][j])

Przykład użycia.

In [11]:
szukanieKc(12, 14)

Znaleziono  19  kart z przedziału [12, 14]
______________________________

> 1. Epsilon  14 kc <

     Sniku
     Losingxcontrol
     wiciu6331
     Calamity
______________________________

> 2. Eta  14 kc <

     Tankomir
     Tymek97517
     Ingsay
______________________________

> 3. Beatrix  14 kc <

     TheBabatunde
______________________________

> 4. Kishibe  14 kc <

     GoldfishBrain
     LargeMonster
     LargeMonster
     Hatchkey
     Krigee
     luniaczek_07
     Kaszalotto
______________________________

> 5. Ruijerd Supardia 13 kc <

     Defx01
     Kaazzz
     aniomak
     MaciekCwirCwir
     Hatchkey
     Katyusha23
     Hatchkey
     Sniku
     Ingsay
     MikiWonsPL
     Basniopisarka
     Samper
     tomoro02
     Matjaz0607
______________________________

> 6. Aladyn  13 kc <

     Ema23
     laros3
     Krasnest
     HerZer
     Jeeda
     Sniku
     Dasshhu
     Tymek97517
______________________________

> 7. Brook  13 kc <

     Kogucik
     JustNikos
     ta

In [12]:
df.head()

,name,ID,kc,users
0,Rias Gremory,23436,56,"[Laudemort, laros3, Kadzee_, DareQPlays]"
1,Ayato Kirishima,68334,12,"[MasterUnit, pekorella, Madcatlover, ReTonZaRR..."
2,Q,50742,1,"[Hurkisq, KrychuXIV, lolcrafter1100, Zielakos,..."
3,Eustass Kid,13620,4,"[paula989, PyraZiemniaczana, Ingsay, dragon8, ..."
4,Musashi,3824,3,"[Sniku, SzymonWidziszowski, GoldfishBrain, Kal..."


In [13]:
n=0
user_list=[]
for i in df['users']:
    n+=1
    for j in i:
        user_list.append(j)
user_list=list(set(user_list))
user_list.sort()

def poUzytkowniku():
    output = []
    current_data = None
    for user in user_list:
        for i in range(len(df)):
            if user in df['users'][i]:
                if current_data:
                    output.append(current_data)
                current_data = {
                    'user': str(user),
                    'cards': {
                        "name": str(df['name'][i]), 
                        'ID': int(df['ID'][i]), 
                        "kc": int(df['kc'][i])
                    }
                } 
    return output

df2 = pd.json_normalize(poUzytkowniku())
df2.set_index(['user', 'cards.name'], inplace=True)
df2.rename_axis(index={'user': 'Użytkownik', 'cards.name': 'Imię postaci'}, inplace=True)
df2.rename(columns={'cards.ID': 'id', 'cards.kc': 'kc'}, inplace=True)
# df2.head()

# Druga funkcja

<div class="alert alert-block alert-info">
    <p>Druga funckcja <b>karty_od_uzytkownika()</b> wymaga trzech parametrów by zadziałać:</p>
 <ul>
    <li>Nazwy uzytkownika</li>
    <li>Nazwa kolumny po której będziemy sortować. Do wyboru są <font color='red'>'Imię postaci'</font>, <font color='red'>'kc'</font> czy może po numerz <font color='red'>'id'</font> </li>
    <li>Sposób sortowania rosnąco czy malejąco, odpowiednio <font color='green'>True</font> lub <font color='green'>False</font></li>
 </ul>


<div class="alert alert-block alert-warning">
<p>Pamiętaj aby nazwa użytkownika oraz nazwa kolumny były umieszczone w cudzysłowiach np. <font color='red'>"JarMar"</font></p>
<p>Zatem wywołanie funkcji wygląda następująco <b>karty_od_uzytkownika(<font color='red'>"JarMar"</font>, <font color='red'>"kc"</font>, <font color='green'>False</font>)</b></p></div>

In [14]:
def karty_od_uzytkownika(name, sort, rosnące):
    original_names = df2.index.levels[0]
    df2.index = df2.index.set_levels([df2.index.levels[0].str.lower(), df2.index.levels[1]])
    name_lower = name.lower()
    
    if name_lower in df2.index.levels[0]:
        original_name = original_names[df2.index.levels[0].tolist().index(name_lower)]
        odpowiedź = df2.loc[name_lower].sort_values(sort, ascending=rosnące).reset_index()
        print(f'Znaleziono {len(odpowiedź)} karty od {original_name}')
        return odpowiedź
    else:
        print(f'Nie znaleziono kart od {name}')
        return pd.DataFrame()


Przykład użycia.

In [15]:
karty_od_uzytkownika('niya', 'kc', False)

Znaleziono 5 karty od Niya


,Imię postaci,id,kc
0,Rika Shinozaki,56711,9
1,Maya,11388,1
2,Cream The Rabbit,13384,0
3,Don Patch,3153,0
4,Bobobo-bo Bo-bobo,3150,0


# Piaskownica

<div class="alert alert-block alert-info">
    <p>Jeżeli jest to twój pierwszy kontakt z językiem Python lepiej nie modyfikuj nic co znajduje się powyżej, gdyż może to spowodować błędy i finalnie przestać działać.</p>
    <p>Jeżeli nie interesuje cię kod, ale chcesz wiedzieć jak działają funkcje przeczytaj opis każdej z nich.</p>
    <p>Poniżej wywołaj dowolną funkcję, by przekonać się kto posiada pożądane przez ciebie karty.</p>
    <p>Powodzenia i miłej zabawy.</p>
</div>